# Required packages:

 + numpy
 + scipy
 + openopt: `pip install OpenOpt DerApproximator FuncDesigner`
 + nlopt: https://nlopt.readthedocs.io/en/latest/NLopt_Installation/

In [ ]:
import numpy as np
from scipy.special import gamma
from scipy.optimize import fsolve, brentq
from scipy.integrate import quad

from IPython.display import display, Math

# Preliminary setup

First, let's input the parameters of the system:

In [ ]:
#Application
R_exp = 7
R=10**R_exp
x_comp = 1. * (10**7) #CPU Cycles
s_raw = 1.* (10**6) #B
s_proc = 1. *(10**4)#B
delta_app = 0.091 #s
kappa = 0.

##Zipf
alpha = 1.0

#Fog
s_cachef_B = 10.0**9 #10.**8 #B
C_compf = 3. * (10**9) #Hz
C_acc = (10./8) *10**9 #Bps
tau_acc = 4. * (10**(-3)) #s

#Cloud
C_compc = 2.*(10**9) #Hz
tau_DB = 1. * (10**(-3)) #s
C_core  = (1./8) * (10**9) #Bps
tau_core = 40. * (10**(-3)) #s

tau_TLSc = tau_core + tau_acc
tau_TLSf = tau_acc

p_n = 0.08 / (10**9) #$/B/h
p_c = 0.033174 #$/2GHz CPU/h
p_s = 0.004446 / (10**9) #$/B/h

Now let's run the next cell to precompute the items popularity and define helper functions for the model

In [ ]:
pop = np.arange(1,R+1)**(-alpha)
pop /= np.sum(pop)

s_cache_f = s_cachef_B / s_proc
def s_cache_c (s_cachec_B): return s_cachec_B / sproc

def che_prob(tc, item_pop): return 1 - np.exp(-item_pop*tc)

def tc_func(C,pop): return fsolve(lambda t: C - np.sum(che_prob(t,pop)), C)[0]

def expected_latency_mg1ps (capacity, arr_rate, job_size=1):
    job_completion_rate = capacity / job_size
    if arr_rate > job_completion_rate:
        return np.inf
    return (1 / job_completion_rate) / (1 - (arr_rate / job_completion_rate))

def variance_latency_mm1ps (capacity, arrival_rate, job_size=1):
    job_completion_rate = capacity / job_size
    #if arr_rate > job_completion_rate:
    #    return np.inf
    lambda_mu = arrival_rate / job_completion_rate 
    denominator = (1/job_completion_rate)**2 * (2+lambda_mu)
    numerator = (1-lambda_mu)**2 * (2-lambda_mu)
    return denominator / numerator

def expected_queue_size_mg1ps (capacity, arrival_rate, job_size=1):
    job_completion_rate = capacity/job_size
    rho = arrival_rate/job_completion_rate
    return rho / (1 - rho)

def expected_queue_size_mginf(capacity, arrival_rate, job_size = 1):
    return arrival_rate / (capacity/job_size)

def variance_exp(parameter):
    return 1 / (parameter*parameter)

def variance_mginf (capacity, job_size=1):
    return variance_exp(capacity/job_size)

def capacity_ccloud (arrival_rate, job_size=1):
    return arrival_rate * job_size / 2


# Finding $\phi$, $k_{LFU}$, and $k_{LRU}$

## The Fog-offload model

First let's create functions for the model

In [ ]:
def arr_rate_fog (phi,arr_rate,pop): return phi*pop*arr_rate
def arr_rate_fog_miss (phi, arr_rate, hit_fog, pop): return phi*pop*(1-hit_fog)*arr_rate
def arr_rate_cloud (phi, arr_rate, pop): return (1-phi)*pop*arr_rate
def arr_rate_cloud_miss (phi, arr_rate, hit_cloud, pop): return (1-phi)*pop*(1-hit_cloud)*arr_rate

def expected_response_time (arr_rate, phi, s_cachec, hit_fog, hit_cloud):
    
    # CLOUD
    lambda_c = np.sum(arr_rate_cloud(phi, arr_rate, pop))

    l_cloud_m = tau_DB + (x_comp / C_compc)
    l_cloud = tau_TLSc + expected_latency_mg1ps(C_core, lambda_c, s_proc) + tau_core \
            + np.sum((1-hit_cloud)*pop*(1-phi))*l_cloud_m

    # FOG
    arr_rate_fog_m = np.sum(arr_rate_fog_miss(phi, arr_rate, hit_fog, pop))

    l_fog_m = tau_TLSf + expected_latency_mg1ps(C_acc, arr_rate_fog_m, s_raw) + tau_acc + \
            expected_latency_mg1ps(C_compf, arr_rate_fog_m, x_comp)
    l_fog = tau_TLSf + np.sum((1-hit_fog)*pop*phi) * l_fog_m
    
    #display(Math(r"\lambda_{{f,m}} = {:2.2f},\, \mathbf E[T_{{f,m}}] = {:2.3f}".format(arr_rate_fog_m, l_fog_m)))
    
    # ALL
    l_acc = expected_latency_mg1ps(C_acc, arr_rate, s_proc) + tau_acc
    
    #display(Math(r"\mathbf E[T_f] = {:2.3f},\, \mathbf E[T_c] = {:2.3f},\, \mathbf E[T_{{acc}}]={:2.3f}".format(l_fog, l_cloud, l_acc)))

    ret = l_acc + np.sum(pop*phi)*l_fog + np.sum(pop*(1-phi))*l_cloud
    #display(Math(r"\mathbf E[T] = {:2.3f}".format(ret)))
    return l_acc + np.sum(pop*phi)*l_fog + np.sum(pop*(1-phi))*l_cloud

def stddev_response_time (arr, phi, s_cachec, hit_fog, hit_cloud):
    # CLOUD
    lambda_c = np.sum(arr_rate_cloud(phi, arr_rate, pop))

    v_cloud_m = variance_mginf (C_compc, x_comp)
    v_cloud = np.sum((1-hit_cloud)*pop*(1-phi)) * v_cloud_m + variance_latency_mm1ps(C_acc, arr_rate, s_proc)
    
    # FOG
    arr_rate_fog_m = np.sum(arr_rate_fog_miss(phi, arr_rate, hit_fog, pop))

    v_fog_m = variance_latency_mm1ps(C_acc, arr_rate_fog_m, s_raw) + \
            variance_latency_mm1ps(C_compf, arr_rate_fog_m, x_comp)
    v_fog = np.sum((1-hit_fog)*pop*phi) * v_fog_m
    
    # ALL
    v_acc = variance_latency_mm1ps(C_acc, arr, s_proc) + tau_acc

    return np.sqrt(v_acc + np.sum(pop*phi)*v_fog + np.sum(pop*(1-phi))*v_cloud)

def cost_network (arr_rate, phi, pop):
    return p_n * s_proc * np.sum(arr_rate_cloud(phi, arr_rate, pop)) * 3600 #To get the hourly cost

def cost_compute(arr_rate, phi, pop, hit_cloud):
    return p_c * expected_queue_size_mginf(C_compc, np.sum(arr_rate_cloud_miss(phi, arr_rate, hit_cloud, pop)), x_comp)

def cost_memory(s_cachec):
    return s_cachec * s_proc * p_s

def cost (arr_rate, phi, pop, hit_cloud, s_cachec):
    return cost_network(arr_rate, phi, pop) + cost_compute(arr_rate, phi, pop, hit_cloud) + cost_memory(s_cachec)

## Numerical resolution

Now let's solve the system for a given $\lambda$ by setting the `arr_rate` variable.



In [ ]:
arr_rate = 1e4

def cost_func(arr_rate, phi, pop, s_cachec):
    tc_cloud = tc_func(s_cachec, pop*(1-phi))
    hit_cloud = che_prob(tc_cloud, pop*(1-phi))
    return cost(arr_rate, phi, pop, hit_cloud, s_cachec)

def constraint_func(arr_rate, phi, s_cachec, kappa, delta, hit_fog=None):
    
    tc_cloud = tc_func(s_cachec, pop*(1-phi))
    hit_cloud = che_prob(tc_cloud, pop*(1-phi))
    if hit_fog is None:
        tc_fog = tc_func(s_cache_f, pop*phi)
        hit_fog = che_prob(tc_fog, pop*phi)
    
    exp = expected_response_time (arr_rate, phi, s_cachec, hit_fog, hit_cloud)
    stddev = stddev_response_time (arr_rate, phi, s_cachec, hit_fog, hit_cloud)
    ret = exp+kappa*stddev
    #display(Math("{:2.3f}={:2.3f}+{}\cdot {:2.3f}".format(ret, exp, kappa, stddev)))
    
    return ret-delta


### Blind
def blind_cost_func(point, arr_rate, pop, *args, **kwargs):
    phi_B,s_cachec = point
    phi = np.zeros(R)+phi_B
    
    return cost_func(arr_rate, phi, pop, s_cachec)

def blind_constraint_func(point, arr_rate, pop, kappa, delta, hit_fog, *args, **kwargs):
    phi_B,s_cachec = point
    phi = np.zeros(R)+phi_B
    
    return constraint_func(arr_rate, phi, s_cachec, kappa, delta, hit_fog)

### LFU
def lfu_cost_func(point, arr_rate, pop, *args, **kwargs):
    klfu,s_cachec = point
    phi = np.zeros(R)
    fklfu = int(klfu)
    phi[np.arange(0,fklfu)] = 1
    if fklfu != R:
        phi[fklfu] = klfu-fklfu
    
    return cost_func(arr_rate, phi, pop, s_cachec)
    
def lfu_constraint_func(point, arr_rate, pop, kappa, delta, *args, **kwargs):
    klfu,s_cachec = point
    phi = np.zeros(R)
    fklfu = int(klfu)
    phi[np.arange(0,fklfu)] = 1
    if fklfu != R:
        phi[fklfu] = klfu-fklfu
    
    return constraint_func(arr_rate, phi, s_cachec, kappa, delta)

    
### LRU
###### DTMC
def qa(t1,t2,pop): return 1. - np.exp(-t1*pop)
def qb(t1,t2,pop): return np.exp(-t2*pop)

def p11(t1,t2,pop): return qa(t1,t2,pop)**2 / (qa(t1,t2,pop) + qb(t1,t2,pop))
def p01(t1,t2,pop): return qa(t1,t2,pop)*(1./(qa(t1,t2,pop)+qb(t1,t2,pop)) -1.)
def p01p11(t1,t2,pop): return p11(t1,t2,pop)+p01(t1,t2,pop)

###### Cost/constraint functions
def lru_cost_func(point, arr_rate, pop, *args, **kwargs):
    klru,s_cachec = point
    tc_filter = tc_func(klru,pop)
    phi = che_prob(tc_filter,pop)
    
    return cost_func(arr_rate, phi, pop, s_cachec)

def lru_constraint_func(point, arr_rate, pop, kappa, delta, *args, **kwargs):
    klru, s_cachec = point
    tc_filter = tc_func(klru,pop)
    phi = che_prob(tc_filter,pop)
    tc_fog = fsolve(lambda t : s_cache_f - np.sum(p01p11(tc_filter,t,pop)), s_cache_f)[0]
    hit_fog = p11(tc_filter, tc_fog, pop)/phi
    
    return constraint_func(arr_rate, phi, s_cachec, kappa, delta, hit_fog)

In a first step, to derive an approximate value of $\phi$, the influence of the cloud cache is overlooked (and $s_{cache,c}$ fixed). $k_{LRU}$ is derived by using Brent's method on $f(t) = \Delta - \mathbf E[T]$

**Warning:** this step might take a while and raise some warnings

In [ ]:
s_cachec = 0

tc_fog = tc_func(s_cache_f, pop)
hit_fog = che_prob(tc_fog, pop)

try:
    phiB = brentq(lambda x : blind_constraint_func((x, s_cachec), arr_rate, pop, kappa, delta_app, hit_fog), 0, 1)
    display(Math(r"\phi_{{B}} = {:2.3f},\; \mathbf{{E}}[T]={:2.2f}\,\mathrm s,\; \Pi={:2.2f}\mathrm \$/\mathrm h".format(phiB, blind_constraint_func((phiB, s_cachec), arr_rate, pop, kappa, delta_app, hit_fog), blind_cost_func((phiB, s_cachec), arr_rate, pop))))
except ValueError:
    print "No solution for the blind problem when ",
    display(Math(r"\lambda={}".format(arr_rate)))
    phiB=0
   
try:
    klfu = brentq(lambda x : lfu_constraint_func((x, s_cachec), arr_rate, pop, kappa, delta_app), 0, R)
    display(Math(r"k_{{LFU}} = {:2.0f},\; \mathbf{{E}}[T]={:2.2f}\,\mathrm s,\; \Pi={:2.2f}\mathrm \$/\mathrm h".format(klfu, lfu_constraint_func((klfu, s_cachec), arr_rate, pop, kappa, delta_app), lfu_cost_func((klfu, s_cachec), arr_rate, pop))))
except ValueError:
    print "No solution for the LFU problem when ",
    display(Math(r"\lambda={}".format(arr_rate)))
    klfu = 142581

try:
    klru = brentq(lambda x : lru_constraint_func((x, s_cachec), arr_rate, pop, kappa, delta_app), 1.5e5, 3.5e5)
    display(Math(r"k_{{LRU}} = {:2.0f},\; \mathbf{{E}}[T]={:2.2f}\,\mathrm s,\; \Pi={:2.2f}\mathrm \$/\mathrm h".format(klru, lru_constraint_func((klru, s_cachec), arr_rate, pop, kappa, delta_app), lru_cost_func((klru, s_cachec), arr_rate, pop))))
except ValueError:
    print "No solution for ",
    display(Math(r"\lambda={}".format(arr_rate)))
    

Now let's use a numerical solver to derive the optimal $s_{cache,c}$ using $k_{LRU}$ as a starting value

In [ ]:
from openopt import NLP

p_blind=NLP(blind_cost_func, [phiB, s_cachec], args=(arr_rate, pop, kappa, delta_app, hit_fog), c=blind_constraint_func, lb=[0,1], ub=[1,R])
opt_res_blind = p_blind.solve("mma")
p_lfu=NLP(lfu_cost_func, [klfu, s_cachec], args=(arr_rate, pop, kappa, delta_app), c=lfu_constraint_func, lb=[1,1], ub=[R,R])
opt_res_lfu = p_lfu.solve("mma")
p_lru=NLP(lru_cost_func, [klru, s_cachec], args=(arr_rate, pop, kappa, delta_app), c=lru_constraint_func, lb=[1,1], ub=[R,R])
opt_res_lru = p_lru.solve("mma")

In [ ]:
phiB,s_cachec_blind = opt_res_blind.xf
display(Math(r"\phi_{{B}} = {:2.3f},\; s_{{cache,c}}={:2.0f},\; \mathbf{{E}}[T]={:2.2f}\,\mathrm s,\; \Pi={:2.2f}\mathrm \$/\mathrm h".format(phiB, s_cachec_blind, blind_constraint_func((phiB, s_cachec_blind), arr_rate, pop, kappa, delta_app, hit_fog), blind_cost_func((phiB, s_cachec_blind), arr_rate, pop))))

klfu,s_cachec_lfu = opt_res_lfu.xf
phi_lfu = np.sum(pop[:int(klfu)])
display(Math(r"k_{{LFU}} = {:2.0f},\; \phi={:2.3f},\; s_{{cache,c}}={:2.0f},\; \mathbf{{E}}[T]={:2.2f}\,\mathrm s,\; \Pi={:2.2f}\mathrm \$/\mathrm h".format(klfu, phi_lfu, s_cachec_lfu, lfu_constraint_func((klfu, s_cachec_lfu), arr_rate, pop, kappa, delta_app), lfu_cost_func((klfu, s_cachec_lfu), arr_rate, pop))))

klru,s_cachec_lru = opt_res_lru.xf
tc_lru = tc_func(int(klru), pop)
hit_lru = che_prob(tc_lru, pop)
phi_lru = np.sum(pop*hit_lru)
display(Math(r"k_{{LRU}} = {:2.0f},\; \phi={:2.3f},\; s_{{cache,c}}={:2.0f},\; \mathbf{{E}}[T]={:2.2f}\,\mathrm s,\; \Pi={:2.2f}\mathrm \$/\mathrm h".format(klru, phi_lru, s_cachec_lru, lru_constraint_func((klru, s_cachec_lru), arr_rate, pop, kappa, delta_app), lru_cost_func((klru, s_cachec_lru), arr_rate, pop))))

In [ ]:
klru,s_cachec_lru = 1.3e5, 2.9e6
tc_lru = tc_func(int(klru), pop)
hit_lru = che_prob(tc_lru, pop)
phi_lru = np.sum(pop*hit_lru)
display(Math(r"k_{{LRU}} = {:2.0f},\; \phi={:2.3f},\; s_{{cache,c}}={:2.0f},\; \mathbf{{E}}[T]={:2.2f}\,\mathrm s,\; \Pi={:2.2f}\mathrm \$/\mathrm h".format(klru, phi_lru, s_cachec_lru, lru_constraint_func((klru, s_cachec_lru), arr_rate, pop, kappa, delta_app), lru_cost_func((klru, s_cachec_lru), arr_rate, pop))))

# From LRU to ABF

## Fitting for $N$

First, let's derive the coefficients $A$ and $B$ in Proposition V.2

In [ ]:
def coef_alpha1 (R):
    A = 1. + (1. + np.log(np.log(R)) - 2 * np.euler_gamma) / (np.log(R) + np.euler_gamma)
    B = 1. / (np.log(R) + np.euler_gamma)
    return A,B

def hralpha_func(R,alpha): return np.sum(np.arange(1,R+1)**(-alpha))

def coef_alpha (R,alpha):
    hralpha = hralpha_func(R,alpha)
    A = R**(1.0/alpha) / ((1.0-alpha) * hralpha)
    B = - gamma(-1.0/alpha) / (alpha**2 * hralpha**(1.0/alpha))
    return A,B

A,B =  coef_alpha1(R) if alpha==1.0 else coef_alpha(R,alpha)

display(Math(r"A={:2.3f},\, B={:2.3f}".format(A,B)))

## From $k_{LRU}$ to $\widehat{k_1}$

Using $\tilde{N}=k_{LRU}$ in Prop. V.1 gives: $k_{LRU} = \frac{1}{\widehat{k_1}} \sum_{\widehat{k_1} \leq k < 2\widehat{k_1}} f(k)$

Let's inverse this equation to find $\widehat{k_1}$.

In [ ]:
def f(k):
    if alpha==1.0:
        return A*k-B*k*np.log(k)
    else:
        return A*k-B*k**(1.0/alpha)

def ntilde_func(k):
    return 1.0 / k * np.sum(f(np.arange(k,2*k)))

def k1_func(n): return fsolve(lambda k : n - ntilde_func(k),n)[0]

k1 = k1_func(klru)

display(Math(r"\widehat{{k_1}}={:2.0f}".format(k1)))

## From $\widehat{k_1}$ to $n_a$

Now, by definition of $\widehat{k_1}$ (proposition V.3), $n_a = f(\widehat{k_1})$

In [ ]:
na = f(k1)

display(Math("n_a={}".format(int(na))))

# Comparing the memory usage

Finally, let's compare the respective memory usage of the LRU and the ABF under the given parameters

## LRU filter
 
First, let's compute the minimal pointer size $s_p = \lceil \log_2 k_{LRU} \rceil$ and derive $m_{LRU} = 3k_{LRU} s_p$

In [ ]:
pointer_size = np.ceil(np.log2(klru))
#print("Pointer size: ", end=None)
display(Math(r"s_p = {:2.0f}\;\mathrm{{bits}}".format(pointer_size)))

m_lru = pointer_size*3*klru
#print("Memory usage:", end=None)
display(Math(r"m_{{LRU}} = {:2.0f}\;\mathrm{{bits}}".format(m_lru)))

## ABF filter

First, we must input the desired false-positive rate $f_p$ in the `f_p` variable.

We can then compute $n_h$ the number of necessary hash-functions and the corresponding memory $m_{ABF}$

In [ ]:
fp = 0.01


def nh_func(fp): return np.ceil(-np.log2(1-np.sqrt(1-fp)))
nh = nh_func(fp)
display(Math("n_h = {:2.0f}".format(nh)))

def m_abf_func(na, nh): return np.ceil(2*na*nh/np.log(2))
m_abf = m_abf_func(na,nh)
display(Math("m_{{ABF}} = {:2.0f}".format(m_abf)))

## LRU - ABF comparison -- ABF implementability


In [ ]:
display(Math(r"\frac{{m_{{LRU}}}}{{m_{{ABf}}}} = {:2.2f}".format(m_lru/m_abf)))

limit = 2**21
if m_abf/2 > limit:
    print("ABF is too big for the Xilinx card's BRAM")
else:
     print("ABF is deployable on the Xilinx card's BRAM")   